<a href="https://colab.research.google.com/github/8johan/retinalseg/blob/main/Copie_de_retinal_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset
import zipfile
from U_net_models import unet ,attunet,SAunet
from tools import imgDataset , create_dir,load_drivedata,load_chasdb1data,data_augment,augmentchasdb1_data,seeding,epoch_time,train,evaluate,calculate_metrics,masl_parse,Diceloss

In [ ]:
with zipfile.ZipFile('/content/retinal.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/chasdb1')


In [ ]:
with zipfile.ZipFile('/content/Drive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Drive')

In [ ]:


np.random.seed(42)
data_path = "/content/drivedata/drivedata/DRIVE/"
(train_x, train_y), (test_x, test_y) = load_data(data_path)
create_dir("new_drivedata/train/image/")
create_dir("new_drivedata/train/mask/")
create_dir("new_drivedata/test/image/")
create_dir("new_drivedata/test/mask/")
augment_data(train_x, train_y, "new_drivedata/train/", augment=True )
augment_data(test_x, test_y, "new_drivedata/test/", augment=False)




In [ ]:
(train_x, train_y), (test_x, test_y) = load_data1("/content/chasdb1/retinal/")
""" Create directories to save the augmented data """
create_dir("new_chasdb1data/train/image/")
create_dir("new_chasdb1data/train/mask/")
create_dir("new_chasdb1data/test/image/")
create_dir("new_chasdb1data/test/mask/")
augment_datachasdb1(train_x, train_y, "new_chasdb1data/train/", augment=True)
augment_datachasdb1(test_x, test_y, "new_chasdb1data/test/", augment=False)

In [ ]:

seeding(42)
""" Directories """
create_dir("mymodel")

""" Load dataset """
train_x = sorted(glob("/content/new_drivedata/train/image/*"))[:140]
train_y = sorted(glob("/content/new_drivedata/train/mask/*"))[:140]

valid_x = sorted(glob("/content/new_drivedata/train/image/*"))[150:200]
valid_y = sorted(glob("/content/new_drivedata/train/mask/*"))[150:200]

data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}\n"
print(data_str)

""" Hyperparameters """
H = 512
W = 512
size = (H, W)
batch_size = 6
num_epochs = 30
lr = 0.001
checkpoint_path = "mymodel/checkpoint.pth"

""" Dataset and loader """
train_dataset = imgDataset(train_x, train_y)
valid_dataset = imgDataset(valid_x, valid_y)

train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

device = torch.device('cuda')
#model =build_unet()
#model=attunet()
#model =build_resunet()
#model=attunet(3,1)
model=unet(3,1)
#model=SAunet(3,1)
#model=unetPlusPlus()
#model=resUnet()
#model=attunet(3,1)
#model=attUnet()
#model=resUnet()
#model=SA_UNet()


model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
#loss_fn = nn.BCEWithLogitsLoss()

#loss_fn=IoULoss()
loss_fn=DiceLoss()
trainingEpoch_loss=[]
validationEpoch_loss=[]
""" Training the model """
best_valid_loss = float("inf")
for epoch in range(num_epochs):
  start_time = time.time()
  train_loss = train(model, train_loader, optimizer, loss_fn, device)
  valid_loss = evaluate(model, valid_loader, loss_fn, device)
  trainingEpoch_loss.append(train_loss)
  if valid_loss < best_valid_loss:
            data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
            print(data_str)
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), checkpoint_path)

  validationEpoch_loss.append(best_valid_loss)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
  data_str += f'\tTrain Loss: {train_loss:.3f}\n'
  data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
  print(data_str)









In [ ]:

""" Seeding """
seeding(42)

""" Folders """
create_dir("results")

""" Load dataset """
test_x = sorted(glob("/content/new_drivedata/test/image/*"))
test_y = sorted(glob("/content/new_drivedata/test/mask/*"))

""" Hyperparameters """
H = 512
W = 512
size = (W, H)
checkpoint_path = "mymodel/checkpoint.pth"

""" Load the checkpoint """
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = unet(3,1)
model = model.to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()

metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0]
time_taken = []

for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
  name = x.split("/")[-1].split(".")[0]
  image = cv2.imread(x, cv2.IMREAD_COLOR)
  x = np.transpose(image, (2, 0, 1))
  x = x/255.0
  x = np.expand_dims(x, axis=0)
  x = x.astype(np.float32)
  x = torch.from_numpy(x)
  x = x.to(device)
  mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
  y = np.expand_dims(mask, axis=0)            ## (1, 512, 512)
  y = y/255.0
  y = np.expand_dims(y, axis=0)               ## (1, 1, 512, 512)
  y = y.astype(np.float32)
  y = torch.from_numpy(y)
  y = y.to(device)
  with torch.no_grad():
    jj

    """ Prediction and Calculating FPS """
    start_time = time.time()
    pred_y = model(x)
    pred_y = torch.sigmoid(pred_y)
    total_time = time.time() - start_time
    time_taken.append(total_time)

    score = calculate_metrics(y, pred_y)
    metrics_score = list(map(add, metrics_score, score))
    pred_y = pred_y[0].cpu().numpy()        ## (1, 512, 512)
    pred_y = np.squeeze(pred_y, axis=0)     ## (512, 512)
    pred_y = pred_y > 0.5
    pred_y = np.array(pred_y, dtype=np.uint8)


  """ Saving masks """
  ori_mask = mask_parse(mask)
  pred_y = mask_parse(pred_y)
  line = np.ones((size[1], 10, 3)) * 128

  cat_images = np.concatenate(
            [image, line, ori_mask, line, pred_y * 255], axis=1
        )
  cv2.imwrite(f"results/{name}""unetdrive"".png", cat_images)

jaccard = metrics_score[0]/len(test_x)
f1 = metrics_score[1]/len(test_x)
recall = metrics_score[2]/len(test_x)
precision = metrics_score[3]/len(test_x)
acc = metrics_score[4]/len(test_x)
print(f"Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Recall: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f}")

fps = 1/np.mean(time_taken)
print("FPS: ", fps)